In [ ]:
"""
GOAL: To classify patents based on abstract text and claims into TWO buckets:
COMPOSITION
METHOD

Good resource: https://skimai.com/fine-tuning-bert-for-sentiment-analysis/
"""

In [ ]:
!pip install transformers==4.10.1 --quiet

     |████████████████████████████████| 2.8 MB 11.5 MB/s 
     |████████████████████████████████| 895 kB 31.2 MB/s 
     |████████████████████████████████| 596 kB 37.9 MB/s 
     |████████████████████████████████| 77 kB 5.9 MB/s 
     |████████████████████████████████| 3.3 MB 41.0 MB/s 


In [ ]:
!pip install pytorch_lightning

     |████████████████████████████████| 582 kB 12.0 MB/s 
     |████████████████████████████████| 398 kB 48.1 MB/s 
     |████████████████████████████████| 136 kB 52.6 MB/s 
     |████████████████████████████████| 1.1 MB 43.6 MB/s 
     |████████████████████████████████| 144 kB 53.5 MB/s 
     |████████████████████████████████| 94 kB 1.1 MB/s 
     |████████████████████████████████| 271 kB 10.7 MB/s 
  Attempting uninstall: typing-extensions
    Found existing installation: typing-extensions 3.10.0.2
    Uninstalling typing-extensions-3.10.0.2:
      Successfully uninstalled typing-extensions-3.10.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow 2.8.0 requires tf-estimator-nightly==2.8.0.dev2021122109, which is not installed.
arviz 0.11.4 requires typing-extensions<4,>=3.7.4.3, but you have typing-extensions 4.1.1 which is incompatible.


In [ ]:
!pip install "torchmetrics<0.7"

     |████████████████████████████████| 332 kB 12.4 MB/s 
  Attempting uninstall: torchmetrics
    Found existing installation: torchmetrics 0.7.3
    Uninstalling torchmetrics-0.7.3:
      Successfully uninstalled torchmetrics-0.7.3


In [ ]:
#testing gpu prescence
import tensorflow as tf
tf.test.gpu_device_name()

'/device:GPU:0'

In [ ]:
import torch
torch.cuda.empty_cache()

In [ ]:
import os
import re
import itertools
import json
import srsly
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import json
from tqdm.auto import tqdm
#pytorch
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from transformers import BertTokenizerFast as BertTokenizer, BertModel as BM, AdamW as Adam

#early stopping
from pytorch_lightning.callbacks.early_stopping import EarlyStopping

#logger
from pytorch_lightning.loggers import TensorBoardLogger

#pytorch scheduler
from transformers import get_linear_schedule_with_warmup
#from torch.optim.lr_scheduler import ExponentialLR

#pytorch lightning - lots of features like model checkpoints, logging, metrics, etc
import transformers
import pytorch_lightning as pl
#from torchmetrics import Accuracy
from torchmetrics.functional import auroc 
#going to create a confusion matrix
from sklearn.metrics import multilabel_confusion_matrix

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)

Mounted at /content/gdrive


In [ ]:
!ls

gdrive	sample_data


In [ ]:
project_folder = "gdrive/MyDrive/Sanah/Patent_Classfication/"
os.chdir(project_folder)

In [ ]:
#creating a training and testing dataset
labeledFile = open('patent_info.json')
#loading data from file
labeledData = json.load(labeledFile)
labeledSize = len(labeledData)
print("Labeled data shape:", len(labeledData))


Labeled data shape: 4577


In [ ]:
#IMP Params
EPOCHS = 12
BATCH_SIZE = 32
MAX_TOKEN_NUM = 128
BERT_MODEL_NAME = "allenai/scibert_scivocab_uncased"

In [ ]:
#conversion into dataframes - training + validation
labels1 = pd.DataFrame([{"METHOD": 1} if labeledData[i]['label'] == "METHOD" else {"METHOD": 0} for i in range(labeledSize)])
labels2 = pd.DataFrame([{"COMPOSITION": 1} if labeledData[i]['label'] == "COMPOSITION" else {"COMPOSITION": 0} for i in range(labeledSize)])
tdText= pd.DataFrame([{"text": labeledData[i]['abstract_text']} for i in range(labeledSize)])
totaldf = pd.concat([tdText, labels1, labels2], axis=1)
totaldf.head()

,text,METHOD,COMPOSITION
0,A method of increasing the bioavailability of ...,1,0
1,The present invention provides a method of tre...,1,0
2,The present invention provides a process for p...,0,1
3,Provided herein are ready-to-use premixed phar...,1,0
4,The method for the treatment chronic uremic pa...,1,0


In [ ]:
COLUMNS = totaldf.columns.tolist()[1:]
COLUMNS

['METHOD', 'COMPOSITION']

In [ ]:
#drop records with incomplete info
totaldf = totaldf.dropna()

In [ ]:
#some useful preprocessing
def text_preprocessing(text):
    """
    - Remove entity mentions (eg. '@united')
    - Correct errors (eg. '&amp;' to '&')
    @param    text (str): a string to be processed.
    @return   text (Str): the processed string.
    """
    # Remove '@name'
    text = re.sub(r'(@.*?)[\s]', ' ', text)

    # Replace '&amp;' with '&'
    text = re.sub(r'&amp;', '&', text)

    # Remove trailing whitespace
    text = re.sub(r'\s+', ' ', text).strip()

    return text

In [ ]:
#use of validation data will help with accuracy prediction 
from sklearn.model_selection import train_test_split
traindf, valdf = train_test_split(totaldf, test_size=0.1)
traindf.shape, valdf.shape

((4118, 3), (458, 3))

In [ ]:
#creating a pytorch dataset class
class PatentDataset(Dataset):
  def __init__(self, data,  tokenizer):
    super().__init__()
    self.tokenizer = tokenizer
    self.data = data
    self.dataLen = len(self.data)

  def __len__(self):
    return self.dataLen

  def __getitem__(self, index: int):
    data_row = self.data.iloc[index]

    medical_text = str(data_row['text'])
    labels = data_row[COLUMNS]

    #creating an encoding 
    encoding = self.tokenizer.encode_plus(
      text_preprocessing(medical_text),
      add_special_tokens=True,
      max_length=MAX_TOKEN_NUM,
      return_token_type_ids=False,
      padding="max_length",
      truncation=True,
      return_attention_mask=True,
      return_tensors='pt',
    )

    return dict(
      medical_text=medical_text,
      input_ids=encoding["input_ids"].flatten(),
      attention_mask=encoding["attention_mask"].flatten(),
      labels=torch.FloatTensor(labels)
    )

In [ ]:
class PatentTextDataModule(pl.LightningDataModule):
  #three datasets - training, testing, validation dataset
  def __init__(self, train_df, val_df, test_df, tokenizer):
    super().__init__()
    self.train_df = train_df
    self.test_df = test_df
    self.val_df = val_df
    self.tokenizer = tokenizer

  #datasets setup - training, validation, testing  
  def setup(self, stage=None):
    self.train_dataset = PatentDataset(
      self.train_df,
      self.tokenizer
    )
    self.val_dataset = PatentDataset(
        self.val_df, 
        self.tokenizer)
    
    self.test_dataset = PatentDataset(
      self.test_df,
      self.tokenizer
    )

  #The num_workers attribute tells the data loader instance how many sub-processes to use for data loading.
  def train_dataloader(self):
    return DataLoader(
      self.train_dataset,
      batch_size= BATCH_SIZE,
      shuffle=True,
      num_workers=2)
    
  def val_dataloader(self):
    return DataLoader(
      self.val_dataset,
      batch_size= BATCH_SIZE,
      shuffle=True,
      num_workers=2)

  def test_dataloader(self):
    return DataLoader(
      self.test_dataset,
      batch_size=BATCH_SIZE,
      num_workers=2)

In [ ]:
# Load the BERT tokenizer
tokenizer = BertTokenizer.from_pretrained(BERT_MODEL_NAME, do_lower_case=True)

Downloading:   0%|          | 0.00/228k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/385 [00:00<?, ?B/s]

In [ ]:
#load a pre-trained BERT model - transfer learning
bert_model = BM.from_pretrained(BERT_MODEL_NAME, return_dict=True)

Downloading:   0%|          | 0.00/442M [00:00<?, ?B/s]

Some weights of the model checkpoint at allenai/scibert_scivocab_uncased were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.predictions.decoder.bias', 'cls.seq_relationship.weight', 'cls.predictions.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
#create a data module with all datasets in place
patent_data_module = PatentTextDataModule(traindf, valdf, valdf, tokenizer)

In [ ]:
class PatentTextLabelClassifier(pl.LightningModule):

  def __init__(self, bert_model, n_classes, criterion, n_training_steps=None, n_warmup_steps=None, fine_tuning=True):
    super().__init__()
    #pre-trained bert_model
    self.bert = bert_model
    # Specify hidden size of BERT, hidden size of our classifier, and number of labels
    D_in, H, D_out = 768, 60, 2
    # Instantiate an one-layer feed-forward classifier
    self.classifier = nn.Sequential(
        nn.Linear(D_in, H),
        nn.ReLU(),
        #nn.Dropout(0.5),
        nn.Linear(H, D_out)
    )
    #how many times step is called
    self.n_training_steps = n_training_steps
    if n_warmup_steps is None:
      self.n_warmup_steps = n_training_steps//5
    else:
      self.n_warmup_steps = n_warmup_steps
    self.criterion = criterion
    self.fine_tuning = fine_tuning

  #the forward function enables us to define how the model goes from input to output
  def forward(self, input_ids, attention_mask, labels=None):
    output = self.bert(input_ids, attention_mask=attention_mask)
    #last layer to a classifier output
    output = self.classifier(output.pooler_output)
    #our activation function
    output = torch.sigmoid(output)    
    loss = 0
    if labels is not None:
        loss = self.criterion(output, labels)
    return loss, output

  def training_step(self, batch, batch_idx):
    #zero out gradients
    self.bert.zero_grad()
    input_ids = batch["input_ids"]
    #attention masks - binary masks to hide the padded indices
    attention_mask = batch["attention_mask"]
    labels = batch["labels"]
    loss, outputs = self(input_ids, attention_mask, labels)
    if (batch_idx % 5000 == 0):
      print({"loss": loss})
    return {"loss": loss, "predictions": outputs, "labels": labels}

  def validation_step(self, batch, batch_idx):
    input_ids = batch["input_ids"]
    attention_mask = batch["attention_mask"]
    labels = batch["labels"]
    loss, outputs = self(input_ids, attention_mask, labels)
    self.log("val_loss", loss, prog_bar=True, logger=True)
    return loss

  def test_step(self, batch, batch_idx):
    input_ids = batch["input_ids"]
    attention_mask = batch["attention_mask"]
    labels = batch["labels"]
    loss, outputs = self(input_ids, attention_mask, labels)
    return loss

  def training_epoch_end(self, outputs):
    #collecting the labels and predictions from the output
    labels = []
    predictions = []
    for output in outputs:
      for output_labels in output["labels"].detach().cpu():
        labels.append(output_labels)
      for output_predictions in output["predictions"].detach().cpu():
        predictions.append(output_predictions)

    labels = torch.stack(labels).int()
    predictions = torch.stack(predictions)
    #return labels, predictions


  def configure_optimizers(self):
    optimizer_grouped_parameters = []
    if self.fine_tuning:  
      param_optimizer = list(bert_model.named_parameters())
      no_decay = ['bias', 'gamma', 'beta']
      optimizer_grouped_parameters = [
          {'params': [p for n, p in param_optimizer if not any(nd in n for nd in no_decay)],
          'weight_decay_rate': 0.01},
          {'params': [p for n, p in param_optimizer if any(nd in n for nd in no_decay)],
          'weight_decay_rate': 0.0}
      ]
    else:
        param_optimizer = list(bert_model.classifier.named_parameters())
        optimizer_grouped_parameters = [{"params": [p for n, p in param_optimizer]}]

    optimizer = Adam(optimizer_grouped_parameters, lr=3e-5, eps=1e-8)
    
    #the scheduler enables us to adjust the learning rate
    scheduler = get_linear_schedule_with_warmup(
      optimizer,
      num_warmup_steps=self.n_warmup_steps,
      num_training_steps=self.n_training_steps
    )
    return dict(
      optimizer=optimizer,
      lr_scheduler=dict(
        scheduler=scheduler,
        interval='step'
      )
    )

In [ ]:
#criterion - measuring the Binary Cross Entropy between the target and the input probabilities
criterion = nn.BCELoss()

In [ ]:
def set_seed(seed_value=42):
    """Set seed for reproducibility.
    """
    np.random.seed(seed_value)
    torch.manual_seed(seed_value)
    torch.cuda.manual_seed_all(seed_value)

In [ ]:
#deciding gpu device
if torch.cuda.is_available():       
    device = torch.device("cuda")
else:
  device = torch.device("cpu")

In [ ]:
#training
#calculating number of train steps
total_training_steps = BATCH_SIZE * EPOCHS

model = PatentTextLabelClassifier(
    bert_model,
  n_classes=len(COLUMNS),
  criterion = criterion,
  n_training_steps=total_training_steps 
)

#move to GPU
model.to(device)

#lightning module training configs
#
trainer = pl.Trainer(
  logger= None,
  checkpoint_callback= None,
  callbacks=[EarlyStopping(monitor="val_loss", mode="min")],
  gpus=1,
  max_epochs=EPOCHS
)


GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


In [ ]:
#set a seed
set_seed(42)

In [ ]:
#fitting 
PATH = "./model_save/my_model"
RETRAIN = False
OVERWRITE = False
if RETRAIN:
  trainer.fit(model, patent_data_module)
  #save model
  if OVERWRITE:
    torch.save(model.state_dict(), PATH)
else:
  model.load_state_dict(torch.load(PATH))

/usr/local/lib/python3.7/dist-packages/pytorch_lightning/callbacks/model_checkpoint.py:608: UserWarning: Checkpoint directory /content/gdrive/MyDrive/Sanah/Patent_Classfication/checkpoints exists and is not empty.
  rank_zero_warn(f"Checkpoint directory {dirpath} exists and is not empty.")
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name       | Type       | Params
------------------------------------------
0 | bert       | BertModel  | 109 M 
1 | classifier | Sequential | 46.3 K
2 | criterion  | BCELoss    | 0     
------------------------------------------
109 M     Trainable params
0         Non-trainable params
109 M     Total params
439.859   Total estimated model params size (MB)


Sanity Checking: 0it [00:00, ?it/s]

/usr/local/lib/python3.7/dist-packages/pytorch_lightning/trainer/connectors/data_connector.py:489: PossibleUserWarning: Your `val_dataloader`'s sampler has shuffling enabled, it is strongly recommended that you turn shuffling off for val/test/predict dataloaders.
  category=PossibleUserWarning,


Training: 0it [00:00, ?it/s]

{'loss': tensor(0.7186, device='cuda:0', grad_fn=<BinaryCrossEntropyBackward0>)}


...

Validation: 0it [00:00, ?it/s]

{'loss': tensor(0.3996, device='cuda:0', grad_fn=<BinaryCrossEntropyBackward0>)}


Validation: 0it [00:00, ?it/s]

{'loss': tensor(0.4685, device='cuda:0', grad_fn=<BinaryCrossEntropyBackward0>)}


Validation: 0it [00:00, ?it/s]

In [ ]:
#evaluation mode
model.eval()
model.freeze()

In [ ]:
#device
device = torch.device("cpu")
#move model
trained_model = model.to(device)

In [ ]:
import torch.nn.functional as F

def bert_predict(model, test_dataloader):
    """Perform a forward pass on the trained BERT model to predict probabilities
    on the test set.
    """
    # Put the model into the evaluation mode. The dropout layers are disabled during
    # the test time.
    model.eval()

    predictions = []
    labels = []

    #wrap the iterable with tqdm
    for item in tqdm(val_dataset):
      _, prediction = trained_model(
        item["input_ids"].unsqueeze(dim=0).to(device), 
        item["attention_mask"].unsqueeze(dim=0).to(device)
      )
      predictions.append(prediction.flatten())
      labels.append(item["labels"].int())

    predictions = torch.stack(predictions).detach().cpu()
    labels = torch.stack(labels).detach().cpu()
    return labels, predictions

In [ ]:
from sklearn.metrics import accuracy_score, roc_curve, auc

def evaluate_roc(probs, y_true):
    """
    - Print AUC and accuracy on the test set
    - Plot ROC
    @params    probs (np.array): an array of predicted probabilities with shape (len(y_true), 2)
    @params    y_true (np.array): an array of the true values with shape (len(y_true),)
    """
    preds = probs[:, 1]
    fpr, tpr, threshold = roc_curve(y_true, preds)
    roc_auc = auc(fpr, tpr)
    print(f'AUC: {roc_auc:.4f}')
       
    # Get accuracy over the test set
    y_pred = np.where(preds >= 0.5, 1, 0)
    accuracy = accuracy_score(y_true, y_pred)
    print(f'Accuracy: {accuracy*100:.2f}%')
    
    # Plot ROC AUC
    plt.title('Receiver Operating Characteristic')
    plt.plot(fpr, tpr, 'b', label = 'AUC = %0.2f' % roc_auc)
    plt.legend(loc = 'lower right')
    plt.plot([0, 1], [0, 1],'r--')
    plt.xlim([0, 1])
    plt.ylim([0, 1])
    plt.ylabel('True Positive Rate')
    plt.xlabel('False Positive Rate')
    plt.show()

In [ ]:
#val_dataloader = patent_data_module.val_dataloader
val_dataset = PatentDataset(valdf, tokenizer)
# Compute predicted probabilities on the test set
labels, predictions = bert_predict(trained_model, val_dataset)
# Evaluate the Bert classifier
evaluate_roc(predictions, labels)

  0%|          | 0/458 [00:00<?, ?it/s]

ValueError: ignored

In [ ]:
# Evaluate the Bert classifier
evaluate_roc(predictions, labels)

In [ ]:
CUTOFF = 0.5
#fscore, precision, recall, etc
y_pred = predictions.numpy()

#true labels, vs predictions - assigned an upper and lower bound
y_true = labels.numpy()
y_pred = np.where(y_pred >= CUTOFF, 1, 0)


In [ ]:
#confusion matrices
cm = multilabel_confusion_matrix(y_true, y_pred)
cm

array([[[216,  28],
        [ 86, 128]],

       [[129,  85],
        [ 26, 218]]])

In [ ]:
def flat_accuracy(preds, labels):
    pred_flat = np.argmax(preds, axis=2).flatten()
    labels_flat = labels.flatten()
    return np.sum(pred_flat == labels_flat) / len(labels_flat)

In [ ]:
flat_accuracy(y_pred, y_true)

In [ ]:
#accuracy
accDict = dict()
for i, label in enumerate(COLUMNS):
  row = cm[i]
  tp = row[0][0]
  tn = row[1][1]
  fp = row[0][1]
  fn = row[1][0]
  accDict[label] = ((tp+tn)/(tp+tn+fp+fn))
accDict
  

{'COMPOSITION': 0.75764192139738, 'METHOD': 0.7510917030567685}

In [ ]:
print("AUROC per label")
for i, name in enumerate(COLUMNS):
  tag_auroc = auroc(predictions[:, i], labels[:, i], pos_label=1)
  print(f"{name}: {tag_auroc}")

In [ ]:
"""
FULL_FINETUNING = True
if FULL_FINETUNING:
    param_optimizer = list(bert_model.named_parameters())
    no_decay = ['bias', 'gamma', 'beta']
    optimizer_grouped_parameters = [
        {'params': [p for n, p in param_optimizer if not any(nd in n for nd in no_decay)],
         'weight_decay_rate': 0.01},
        {'params': [p for n, p in param_optimizer if any(nd in n for nd in no_decay)],
         'weight_decay_rate': 0.0}
    ]
else:
    param_optimizer = list(bert_model.classifier.named_parameters())
    optimizer_grouped_parameters = [{"params": [p for n, p in param_optimizer]}]

optimizer = Adam(
    optimizer_grouped_parameters,
    lr=3e-5,
    eps=1e-8
)
"""